In [ ]:
%pip install -r requirements.txt

In [ ]:
#librerias
import cv2
import tensorflow_hub as hub
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
import os
import glob


In [ ]:

# video
cam = cv2.VideoCapture("IMG_1196.mov")

currentframe = 0

while (True):
    ret, frame = cam.read()
    if ret:
        name = f'frames/frame{currentframe}.jpg'
        cv2.imwrite(name, frame)
        currentframe += 1
    else:
        break

cam.release()
cv2.destroyAllWindows()

# pre-trained styleGAN
model = hub.load(
    'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')


def load_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = img[tf.newaxis, :]
    return img

# %%
style_image = load_image('style1.jpeg')
# plt.imshow(np.squeeze(style_image))
# plt.show()

frames_images = os.listdir('frames')
for frame in range(len(frames_images)):
    content_image = load_image(f'frames/frame{frame}.jpg')
    # plt.imshow(np.squeeze(content_image))
    # plt.show()
    stylized_image = model(tf.constant(content_image), tf.constant(style_image))[0]
    # plt.imshow(np.squeeze(stylized_image))
    # plt.show()
    cv2.imwrite(f'styled_frames/generated_frame{frame}.jpg',
                cv2.cvtColor(np.squeeze(stylized_image)*255, cv2.COLOR_BGR2RGB))

img_array = []
for generated_frame in range(len(frames_images)):
    img = cv2.imread(f'styled_frames/generated_frame{generated_frame}.jpg')
    height, width, layers = img.shape
    size = (width, height)
    img_array.append(img)

# out = cv2.VideoWriter('project.avi', cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
out = cv2.VideoWriter('styled_video.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 15, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [1]:
import cv2
import os

frames_images = os.listdir('styled_frames')

img_array = []
for generated_frame in range(len(frames_images)):
    img = cv2.imread(f'styled_frames/generated_frame{generated_frame}.jpg')
    height, width, layers = img.shape
    size = (width, height)
    img_array.append(img)

# out = cv2.VideoWriter('project.avi', cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
out = cv2.VideoWriter('styled_video.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 15, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
